In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [1]:
%cd '/content/drive/My Drive/thesis'
import config_16x15_seq
%cd '/content/drive/My Drive/thesis/config_16x15_seq'

/content/drive/My Drive/thesis
/content/drive/My Drive/thesis/config_16x15_seq


In [3]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        print('ERROR!')
    else:
        print('Found GPU at: {}'.format(device_name))
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print('TPU address is', tpu_address)
    with tf.Session(tpu_address) as session:
        devices = session.list_devices()
        print('TPU devices:')
        pprint.pprint(devices)



Found GPU at: /device:GPU:0


In [2]:
from config_16x15_seq.builder import *
from tqdm import tqdm
from keras.callbacks import BaseLogger, History, CallbackList
from scipy.ndimage import gaussian_filter
import copy

MU_training = MU['training']
MU_norm_training = MU_norm['training']
PHI_meas_training = PHI_meas['training']
MUa_training = MUa['training']
MUsp_training = MUsp['training']
freq_training = freq['training']
d_training = d['training']
lr = 0.0002
beta_1 = 0.5
# clip_value = 0.01
optimizer = Adam(lr=lr, beta_1=beta_1)
# optimizer = RMSprop(lr=lr)
generator = primary_net()
print('Generator model summary:')
generator.summary()
discriminator = secondary_net()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[custom_binary_accuracy])
print('Discriminator model summary:')
discriminator.summary()
discriminator.trainable = False
# inputs = Input((PHI_meas_training.shape[1],))
outputs = generator.outputs
outputs.append(discriminator(outputs[:2]))
gan = Model(inputs=generator.inputs, outputs=outputs)
loss = ['mse', 'mse', 'mse', 'mse', 'mse', 'mse', 'binary_crossentropy']
loss_weights = [0, 0, 5, 5, 1e4, 1, 1]
metrics = ['mae', 'mse', 'mape', 'msle', 'logcosh', 'cosine', custom_binary_accuracy]
gan.compile(loss=loss, loss_weights=loss_weights, optimizer=optimizer, metrics=metrics)
print('Generative adversarial network model summary:')
gan.summary()
history = History()
logger = BaseLogger(stateful_metrics=['discriminator_' + s for s in discriminator.stateful_metric_names] +
                                     ['gan_' + s for s in gan.stateful_metric_names])
# checkpoint = callbacks.ModelCheckpoint('model_test{epoch:02d}.h5', verbose=1)
time_history = TimeHistory()
callbacks = [logger, time_history, history]
out_labels = ['discriminator_' + s for s in discriminator.metrics_names] + ['gan_' + s for s in gan.metrics_names]
callback_metrics = copy.copy(out_labels) + ['val_' + n for n in out_labels]
callbacks = CallbackList(callbacks)
# callbacks = [TimeHistory(), EarlyStopping(monitor='loss', min_delta=0.1, patience=100,
#                                           restore_best_weights=True, verbose=1)]
# early_stop = EarlyStopping(patience=200, verbose=1)
choice = np.zeros(data_size, dtype='uint8')
choice[np.random.choice(data_size, val_size, replace=False)] = 1
MU_training_train = MU_training[choice == 0]
MU_training_val = MU_training[choice == 1]
MU_norm_training_train = MU_norm_training[choice == 0]
MU_norm_training_val = MU_norm_training[choice == 1]
PHI_meas_training_train = PHI_meas_training[choice == 0]
PHI_meas_training_val = PHI_meas_training[choice == 1]
freq_training_train = freq_training[choice == 0]
freq_training_val = freq_training[choice == 1]
d_training_train = d_training[choice == 0]
d_training_val = d_training[choice == 1]
MUa_train = MUa_training[choice == 0]
MUa_val = MUa_training[choice == 1]
MUsp_train = MUsp_training[choice == 0]
MUsp_val = MUsp_training[choice == 1]
# choice = data_cat
batch_size = 32
prob_param = 20.0
prob_param2 = 1.0
# label_softness = 0.1
# metrics_eval_n = 50
# metrics_eval_step = 20
hard_valid = np.ones(val_size)
hard_fake = np.zeros(val_size)


# soft_label = False
# augment = True
# min_img_loss_avg = np.inf
# min_dis_loss_avg = np.inf
# img_loss_baseline_factor = 1.1
# img_loss_threshold = 1.2
# img_loss_threshold2 = 0.6
# prob_augment = 0.5
# gan_d_acc = [0.5, 0.5]
# init_train = 100

N_count_train = np.zeros_like(N_count)
n = 0
for i in range(len(N_count)):
    ni = N_count[i]
    N_count_train[i] = np.where(choice[n:(n + ni)] == 0)[0].size
    n += ni
index_array = np.arange(train_size)
index_array2 = np.arange(train_size)
ia = np.arange(train_size)
n = N_count_train[0]
np.random.shuffle(index_array[:n])
np.random.shuffle(index_array2[:n])
for i in range(1, len(N_count_train) - 1):
    ni = N_count_train[i]
    np.random.shuffle(index_array[n:(n + ni)])
    np.random.shuffle(index_array2[n:(n + ni)])
    n += ni
np.random.shuffle(index_array[n:])
np.random.shuffle(index_array2[n:])


def run_epoch(epochs):
    callbacks.set_params({
        'batch_size': batch_size,
        'epochs': epochs,
        'steps': None,
        'samples': train_size,
        'verbose': 2,
        'do_validation': True,
        'metrics': callback_metrics,
    })
    callbacks.on_train_begin()
    for epoch in range(epochs):
        for m in discriminator.stateful_metric_functions:
            m.reset_states()
        for m in gan.stateful_metric_functions:
            m.reset_states()
        callbacks.on_epoch_begin(epoch)
        epoch_logs = {}
        progress_bar = None
        # np.random.shuffle(ia)
        n = N_count_train[0]
        np.random.shuffle(ia[:n])
        for i in range(1, len(N_count_train) - 1):
            ni = N_count_train[i]
            np.random.shuffle(ia[n:(n + ni)])
            n += ni
        np.random.shuffle(ia[n:])
        num_batches = (train_size + batch_size - 1) // batch_size
        # if epoch > 0 and epoch % metrics_eval_step == 0:
        # hist_avg = {}
        # for k, v in history.history.items():
        #     if k.startswith('val_'):
        #         l = k[4:]
        #     else:
        #         l = k
        #     if l in logger.stateful_metrics:
        #         hist_avg[k] = v[-1]
        #     else:
        #         hist_avg[k] = np.average(v[-metrics_eval_n:])
        # img_loss_avg = hist_avg['val_gan_' + gan.metrics_names[0]] - hist_avg['val_gan_' + gan.metrics_names[7]]
        # dis_loss_avg = hist_avg['val_discriminator_' + discriminator.metrics_names[0]]
        # min_img_loss = np.min(history.history['val_gan_' + gan.metrics_names[0]]) \
        #                - np.min(history.history['val_gan_' + gan.metrics_names[7]])
        # if img_loss_avg > min_img_loss_avg * img_loss_baseline_factor:
        #     if not augment:
        #         augment = True
        #     elif img_loss_avg >= img_loss_threshold and min_img_loss >= img_loss_threshold2:
        #         soft_label = True
        # elif dis_loss_avg > min_dis_loss_avg:
        #     if augment:
        #         augment = False
        #     elif img_loss_avg < img_loss_threshold:
        #         soft_label = False
        # img_loss_avg = np.average(history.history['val_gan_' + gan.metrics_names[0]][-metrics_eval_n:])\
        #                - np.average(history.history['val_gan_' + gan.metrics_names[7]][-metrics_eval_n:])
        # if soft_label and img_loss_avg < img_loss_threshold:
        #     soft_label = False
        # if np.random.random() > 0.5:
        #     augment = not augment
        # min_img_loss_avg = min(img_loss_avg, min_img_loss_avg)
        # min_dis_loss_avg = min(dis_loss_avg, min_dis_loss_avg)
        # if epoch >= init_train:
        #     if gan_d_acc[0] < 0.5 or gan_d_acc[1] < 0.5:
        #         prob_augment = 0.8
        #     else:
        #         prob_augment = 0.5
        #     if augment == (np.random.random() < prob_augment):
        #         augment = not augment
        batch_end = 0
        # prob_augment = np.random.random()
        for batch_index in range(num_batches):
            augment = np.random.random() < 0.5
            batch_start = batch_end
            batch_end = min(train_size, batch_end + batch_size)
            batch_ids = index_array[ia[batch_start:batch_end]]
            batch_logs = {'batch': batch_index, 'size': len(batch_ids)}
            callbacks.on_batch_begin(batch_index, batch_logs)
            MU_gt = [MU_training_train[batch_ids, 0], MU_training_train[batch_ids, 1]]
            MU_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_gt]
            in_gen = [PHI_meas_training_train[batch_ids], freq_training_train[batch_ids], d_training_train[batch_ids]]
            if augment:
                noise_param = np.random.random(len(batch_ids))
                blur_param = np.random.random(len(batch_ids))
                noise_param *= blur_param
                MU_gt_aug = [np.copy(im) for im in MU_gt]
                for i, bi in enumerate(batch_ids):
                    temp = mask_image(MU_gt_aug[0][i, 0], MUa_train[bi])
                    temp = temp + temp * np.random.normal(scale=noise_param[i] * 0.5, size=temp.shape)
                    temp = gaussian_filter(temp, blur_param[i] * 5.0)
                    MU_gt_aug[0][i, 0] = mask_image(temp, 0.0)
                    temp = mask_image(MU_gt_aug[1][i, 0], MUsp_train[bi])
                    temp = temp + temp * np.random.normal(scale=noise_param[i] * 0.5, size=temp.shape)
                    temp = gaussian_filter(temp, blur_param[i] * 5.0)
                    MU_gt_aug[1][i, 0] = mask_image(temp, 0.0)
                # temp = np.clip(blur_param, prob_param2 / prob_param, 1.0 - prob_param2 / prob_param)
                valid = 0.95 - 0.05 * np.random.random(len(batch_ids))
                # valid = 1.0 - label_softness * np.random.beta(temp * prob_param, (1.0 - temp) * prob_param)
                # if soft_label:
                #     fake = label_softness * np.random.random(len(batch_ids))
                #     valid = 1.0 - label_softness * np.random.beta(temp * prob_param, (1.0 - temp) * prob_param)
                # else:
                #     fake = hard_fake[:len(batch_ids)]
                #     valid = hard_valid[:len(batch_ids)] - label_softness
            else:
                MU_gt_aug = MU_gt
                valid = hard_valid[:len(batch_ids)] - 0.05
                # valid = 1.0 - 0.05 * np.random.random(len(batch_ids))
                # valid = 1.0 - label_softness * np.random.random(len(batch_ids))
                # valid = 1.0 - label_softness * np.random.beta(prob_param2,
                #                                               prob_param - prob_param2, size=len(batch_ids))
                # if soft_label:
                #     fake = label_softness * np.random.random(len(batch_ids))
                #     valid = 1.0 - label_softness * np.random.beta(
                #         prob_param2, prob_param - prob_param2, size=len(batch_ids))
                # else:
                #     fake = hard_fake[:len(batch_ids)]
                #     valid = hard_valid[:len(batch_ids)] - label_softness
            # fake = hard_fake[:len(batch_ids)]
            fake = 0.05 * np.random.random(len(batch_ids))
            # if epoch < init_train:
            #     valid -= label_softness * np.random.random(len(batch_ids)) * (init_train - epoch) / init_train
            #     fake += label_softness * np.random.random(len(batch_ids)) * (init_train - epoch) / init_train
            d_loss1 = discriminator.train_on_batch(MU_gt_aug, valid)
            # d_loss1 = discriminator.train_on_batch([np.concatenate([im1, im2], axis=0)
            #                                        for im1, im2 in zip(MU_gt_aug, MU_gt)],
            #                                        np.concatenate([valid, hard_valid[:len(batch_ids)]], axis=0),
            #                                        sample_weight=np.ones(2 * len(batch_ids)) * 0.5)
            d_loss2 = discriminator.train_on_batch(generator.predict(in_gen)[:2], fake)
            # d_loss = discriminator.train_on_batch([np.concatenate([im1, im2], axis=0)
            #                                        for im1, im2 in zip(MU_gt_aug, generator.predict(in_gen)[:2])],
            #                                       np.concatenate([valid, fake], axis=0))
            d_loss = np.zeros(len(discriminator.metrics_names))
            for i, l in enumerate(discriminator.metrics_names):
                if l in discriminator.stateful_metric_names:
                    d_loss[i] = d_loss2[i]
                else:
                    d_loss[i] = 0.5 * (d_loss1[i] + d_loss2[i])
            # for l in discriminator.layers:
            #     weights = l.get_weights()
            #     weights = [np.clip(w, -clip_value, clip_value) for w in weights]
            #     l.set_weights(weights)
            batch_ids = index_array2[ia[batch_start:batch_end]]
            MU_gt = [MU_training_train[batch_ids, 0], MU_training_train[batch_ids, 1]]
            MU_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_gt]
            in_gen = [PHI_meas_training_train[batch_ids], freq_training_train[batch_ids], d_training_train[batch_ids]]
            MU_norm_gt = [MU_norm_training_train[batch_ids, 0], MU_norm_training_train[batch_ids, 1]]
            MU_norm_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_norm_gt]
            g_loss = gan.train_on_batch(in_gen, [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1],
                                                 MUa_train[batch_ids], MUsp_train[batch_ids],
                                                 hard_valid[:len(batch_ids)]])
            for l, o in zip(out_labels, np.concatenate([d_loss, g_loss])):
                batch_logs[l] = o
            callbacks.on_batch_end(batch_index, batch_logs)
            if progress_bar is None:
                progress_bar = tqdm(total=train_size, desc='Epoch {}/{}'.format(epoch + 1, epochs), unit='samples')
            progress_bar.set_postfix(train_on_batch=('[D loss: [%g %g], acc.: [%.2f%% %.2f%%]] [G total_loss: %g, ' +
                                                     'img_loss: %g, d_loss: %g, d_acc.: %.2f%%]') %
                                                    (d_loss1[0], d_loss2[0], 100 * d_loss1[1], 100 * d_loss2[1],
                                                     g_loss[0],
                                                     g_loss[0] - g_loss[7], g_loss[7], 100 * g_loss[-1]),
                                     augment=augment)
            progress_bar.update(len(batch_ids))
        MU_gt = [MU_training_val[:, np.array([0])], MU_training_val[:, np.array([1])]]
        d_val_loss1 = discriminator.evaluate(MU_gt, hard_valid, batch_size=batch_size, verbose=0)
        d_val_loss2 = discriminator.evaluate(generator.predict([PHI_meas_training_val, freq_training_val,
                                                                d_training_val])[:2], hard_fake, batch_size=batch_size,
                                             verbose=0)
        # d_val_loss = discriminator.evaluate([np.concatenate([im1, im2], axis=0)
        #                                      for im1, im2 in zip(MU_gt,
        #                                                          generator.predict(PHI_meas_training_val)[:2])],
        #                                     np.concatenate([hard_valid, hard_fake], axis=0), verbose=0)
        d_val_loss = np.zeros(len(discriminator.metrics_names))
        for i, l in enumerate(discriminator.metrics_names):
            if l in discriminator.stateful_metric_names:
                d_val_loss[i] = d_val_loss2[i]
            else:
                d_val_loss[i] = 0.5 * (d_val_loss1[i] + d_val_loss2[i])
        MU_norm_gt = [MU_norm_training_val[:, np.array([0])], MU_norm_training_val[:, np.array([1])]]
        progress_bar.set_postfix(validate_on_epoch='[D loss: [%g %g], acc.: [%.2f%% %.2f%%]] validating generator..' %
                                                   (d_val_loss1[0], d_val_loss2[0], 100 * d_val_loss1[1],
                                                    100 * d_val_loss2[1]))
        g_val_loss = gan.evaluate([PHI_meas_training_val, freq_training_val, d_training_val],
                                  [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1], MUa_val, MUsp_val, hard_valid],
                                  batch_size=batch_size, verbose=0)
        for l, o in zip(out_labels, np.concatenate([d_val_loss, g_val_loss])):
            epoch_logs['val_' + l] = o
        # num_batches = (val_size + batch_size - 1) // batch_size
        # for batch_index in range(num_batches):
        #     batch_start = batch_index * batch_size
        #     batch_end = min(val_size, (batch_index + 1) * batch_size)
        #     MU_gt = [MU_training_val[batch_start:batch_end, 0], MU_training_val[batch_start:batch_end, 0]]
        #     MU_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_gt]
        #     d_loss1 = discriminator.test_on_batch(MU_gt, hard_valid[:(batch_end - batch_start)])
        #     in_gen = PHI_meas_training_val[batch_start:batch_end]
        #     d_loss2 = discriminator.test_on_batch(generator.predict(in_gen)[:2], hard_fake[:(batch_end - batch_start)])
        #     d_loss = np.zeros(len(discriminator.metrics_names))
        #     for i, l in enumerate(discriminator.metrics_names):
        #         if l in discriminator.stateful_metric_names:
        #             d_loss[i] = d_loss2[i]
        #         else:
        #             d_loss[i] = 0.5 * (d_loss1[i] + d_loss2[i])
        #     MU_norm_gt = [MU_norm_training_val[batch_start:batch_end, 0],
        #                   MU_norm_training_val[batch_start:batch_end, 1]]
        #     MU_norm_gt = [im.reshape(im.shape[:1] + (1,) + im.shape[1:]) for im in MU_norm_gt]
        #     g_loss = gan.test_on_batch(in_gen, [MU_gt[0], MU_gt[1], MU_norm_gt[0], MU_norm_gt[1],
        #                                       MUa_val[batch_start:batch_end], MUsp_val[batch_start:batch_end],
        #                                       hard_valid[:(batch_end - batch_start)]])
        #     for l, o in zip(out_labels, np.concatenate([d_loss, g_loss])):
        #         if l in logger.stateful_metrics:
        #             epoch_logs['val_' + l] = o
        #         else:
        #             if 'val_' + l in epoch_logs:
        #                 epoch_logs['val_' + l] += o * (batch_end - batch_start)
        #             else:
        #                 epoch_logs['val_' + l] = o * (batch_end - batch_start)
        #     pbar.set_postfix(validate_on_batch=('[D loss: %g, acc.: %.2f%%] [G total_loss: %g, img_loss: %g, ' +
        #                                         'd_loss: %g, d_acc.: %.2f%%]') %
        #                                        (d_loss[0], 100 * d_loss[1], g_loss[0], g_loss[0] - g_loss[7],
        #                                         g_loss[7], 100 * g_loss[-1]), soft_label=soft_label, augment=augment)
        #     pbar.update(batch_end - batch_start)
        # for l in out_labels:
        #     if l not in logger.stateful_metrics:
        #         epoch_logs['val_' + l] /= val_size
        callbacks.on_epoch_end(epoch, epoch_logs)
        # gan_d_acc[0] = epoch_logs[out_labels[-1]]
        # gan_d_acc[1] = epoch_logs['val_' + out_labels[-1]]
        progress_bar.set_postfix(train=('[D loss: %g, acc.: %.2f%%] [G total_loss: %g, img_loss: %g, d_loss: %g, ' +
                                        'd_acc.: %.2f%%]') %
                                       (epoch_logs[out_labels[0]], 100 * epoch_logs[out_labels[1]],
                                        epoch_logs[out_labels[2]],
                                        epoch_logs[out_labels[2]] - epoch_logs[out_labels[9]],
                                        epoch_logs[out_labels[9]],
                                        100 * epoch_logs[out_labels[-1]]),
                                 validation=('[D loss: %g, acc.: %.2f%%] [G total_loss: %g, img_loss: %g, ' +
                                             'd_loss: %g, d_acc.: %.2f%%]') %
                                            (epoch_logs['val_' + out_labels[0]],
                                             100 * epoch_logs['val_' + out_labels[1]],
                                             epoch_logs['val_' + out_labels[2]],
                                             epoch_logs['val_' + out_labels[2]] - epoch_logs['val_' + out_labels[9]],
                                             epoch_logs['val_' + out_labels[9]],
                                             100 * epoch_logs['val_' + out_labels[-1]]))
        progress_bar.close()
    callbacks.on_train_end()
    hist_data = {
        'train_start': time_history.train_time_start,
        'train_time': time_history.train_time,
        'epoch_time': time_history.times,
        'history': history.history,
        'choice': np.where(choice == 1)[0].tolist()
    }
    return hist_data


hist_data = run_epoch(100)

gan.save('model46log_100.h5')
generator.save('model46log_100_gen.h5')
discriminator.save('model46log_100_dis.h5')
with open('model_hist_data46.0log.json', 'w') as f:
    json.dump(hist_data, f)


Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





Generator model summary:
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 7)      0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 6)      0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
_____________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Epoch 1/100:   0%|          | 0/8000 [00:00<?, ?samples/s, augment=1, train_on_batch=[D loss: [0.695669 0.842798], acc.: [62.50% 28.12%]] [G total_loss: 8.04025, img_loss: 7.38172, d_loss: 0.658526, d_acc.: 62.50%]]/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Epoch 100/100: 100%|██████████| 8000/8000 [03:31<00:00, 42.04samples/s, train=[D loss: 0.178141, acc.: 99.86%] [G total_loss: 2.61244, img_loss: 2.50075, d_loss: 0.111695, d_acc.: 98.95%], validation=[D loss